# 📖 Overview

We commonly work with predictions from the final layer of a neural network in deep learning tasks. In some circumstances, the outputs of intermediate layers may also be of relevance (especially in **semantic segmentations tasks**). It may be difficult to extract intermediate characteristics from the network, whether we wish to extract data embeddings or evaluate what prior layers have learned.

This Notebook demonstrates how to use PyTorch's forward hook feature to harvest intermediate activations from any layer of a deep learning model. The simplicity and ability to extract features without having to run the inference twice is a significant benefit of this method, which only requires a single forward pass through the model to save many outputs.

# 📖 Why are intermediate features necessary?

Many applications benefit from the extraction of intermediate activations (also known as features). The outputs of intermediate CNN layers are commonly used to demonstrate the learning process and illustrate visual features discriminated by the model on different layers in computer vision challenges. Another common application is extracting intermediate outputs to build image or text embeddings, which can be used to detect duplicate items, include as input characteristics in a traditional ML model, show data clusters, and more. The outputs of intermediary layers can also be utilized to compress data into a smaller-sized vector carrying the data representation when dealing with Encoder-Decoder architectures. Intermediate activations can be useful in a variety of other situations. So let's talk about how to get them!

# 📖 How to extract intermediate features?

To extract activations from intermediate layers, we'll need to create a forward hook in our neural network for the layers we're interested in and use inference to store the relevant outputs.


# 📖 Model

To extract anything from a neural network, we must first put it up, correct? We create a basic Efficientnet b0 model with a two-node output layer in the cell below. The model is instantiated with the timm library, however feature extraction will work with any neural network constructed in PyTorch.

We also print out our network's architecture. As you can see, our image passes through numerous intermediary layers during a forward pass before becoming a two-number output. The names of the layers should be written down since they will be required by a feature extraction algorithm.

## Imports

In [ ]:
import numpy as np
import pandas as pd

# pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# timm
!pip install timm
import timm

# augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

import cv2
import os
from glob import glob

device = torch.device('cuda')

In [ ]:
model    = timm.create_model(model_name = 'efficientnet_b0', pretrained = True)
model.fc = nn.Linear(512, 2)
model

# 📖 Feature extraction

Feature extraction may be implemented in two easy steps:

1. Registering a forward hook on a certain network layer.
2. To extract features from that layer, use put model in inference.

To begin, we must create a helper function that will add a hook. When a forward or backward call to a certain layer is made, a hook is just a command that is executed. This [site](https://blog.paperspace.com/pytorch-hooks-gradient-clipping-debugging/) will tell you all you need to know about hooks.

We're looking for a forward hook that just replicates the layer outputs, delivers them to the CPU, and stores them to a dictionary object called features in our setup.

In the cell below, the hook is defined. The dictionary key under which we will keep our intermediate activations is specified by the name parameter in get features().

In [ ]:
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()
    return hook

We may use the .register forward hook() method to register a hook after the helper function has been created. Any layer of the neural network can be used as a hook.

Because we're using a CNN, collecting features from the final convolutional layer might be beneficial for obtaining  embeddings. As a result, we're registering a hook for the (global pool) outputs. We could also use model.layer1[1].act2 to extract features from an earlier layer and put them in the features dictionary under a different name. We can technically register many hooks (one for each layer of interest) with this technique, but for the sake of this example, we'll simply maintain one.

In [ ]:
model.global_pool.register_forward_hook(get_features('feats'))

We're all set to extract features now! The good thing about hooks is that we can now conduct inference as normal while simultaneously receiving many outputs:

1. the last layer's outputs
2. each layer's outputs using a registered hook

When we run the model, the feature extraction happens automatically during the forward pass (inputs). We only need to put the following in our inference loop to save intermediate features and concatenate them across batches:

* Make a placeholder list called **Feat = []**. All batch intermediate outputs will be stored in this list.

* Make a placeholder dict with **features = {}** This dictionary will be used to store intermediate outputs from each batch.

* Extract batch features to **features** iteratively, send them to the CPU, and add them to the **Feat** list.


In [ ]:
# # placeholders
# PREDS = []
# FEATS = []

# # placeholder for batch features
# features = {}

# # loop through batches
# for idx, inputs in enumerate(test_dataloader):

#     # move to device
#     inputs = inputs.to(device)
       
#     # forward pass [with feature extraction]
#     preds = model(inputs)
    
#     # add feats and preds to lists
#     PREDS.append(preds.detach().cpu().numpy())
#     FEATS.append(features['feats'].cpu().numpy())

#     # early stop
#     if idx == 9:
#         break

# Thanks for your attention